In [2]:
import random
import numpy as np
import pandas as pd
from pathlib import Path

PROJECT_PATH = Path("D:/Knee_Flexion_auditory_tactile_feedback/Analysis/results")
angles = [1, 2, 3, 4]
number_of_angles = len(angles)

n = {'Pre_test': 5,
      'RT-1' : 6,
      'Feedback-1' : 24,
      'RT-2' : 6,
      'Feedback-2' : 24,
      'RT-3' : 6,
      'Feedback-3' : 24,
      'RT-Final' : 6,
      'RT-15 min' : 6,
      'RT-24 hours': 6,
      'Transfer': 6}

conf = pd.read_csv(PROJECT_PATH / '_conf.csv')
conf = conf[conf.process==1]
conf

,participant,age,height,mass,sex,leg,feedback_group,conf_file,process
0,P0,30,167,61,1,L,T,D:\Knee_Flexion_auditory_tactile_feedback\Anal...,1
1,N0,30,167,61,1,L,T,D:\Knee_Flexion_auditory_tactile_feedback\Anal...,1


In [3]:
for iparticipant in conf.participant:
    number_of_angles = len(angles)
    number_tests = np.sum(list(n.values())) + len(n.keys())
    groups = np.zeros((number_of_angles, int(number_tests)))
    irow = 0
    header_row_indices = np.zeros(len(n.keys())).astype(int)
    groups.T[0] = np.ones(4)*10
    for i, igroup in enumerate(list(n.keys())):
        number_of_angles = len(angles)
        if i+1!=len(n.keys()):
            irow += n[igroup] + 1
            header_row_indices[i+1] = irow
            groups.T[irow] = np.ones(4)*10
        else:
            irow = groups.shape[1]

        number_participants = 4*n[igroup]
        participants = np.arange(1, number_participants+1, 1).tolist()

        i_group = 0

        while number_participants > 0 and number_of_angles > 1:
            team = random.sample(participants, n[igroup])
            groups[i_group, header_row_indices[i]+1 : irow] = np.sort(team)
            for x in team:
                participants.remove(x)
                number_participants -= 1
            number_of_angles -= 1
            i_group += 1
        groups[-1, header_row_indices[i]+1 : irow] = participants

    groups = groups.astype(int).T
    output_table = pd.DataFrame(groups)
    for i, irow in enumerate(header_row_indices):
        output_table.loc[[irow]]=[list(n.keys())[i], '','','']
    output_table.columns = angles
    
    output_path = PROJECT_PATH / iparticipant / 'angle_order_rec.csv'
    output_table.to_csv(output_path, index=False)